In [1]:
import os, random, time
import prophet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from datetime import date
#import codecs

Importing plotly failed. Interactive plots will not work.


In [3]:
f_list = os.listdir('./data/')
f_list.sort()
print(f_list)
dt_info = np.vstack((np.column_stack((np.repeat(2020,53),np.arange(1,54))), np.column_stack((np.repeat(2021,13),np.arange(1,14)))))

['1_P34_판매계획_2019_10_11_12월.csv', '2_P34_판매계획_2020전체.csv', '3_P34_판매계획_2021_01_02_03월.CSV', '4_P34_생산실적_2020.csv', '5_P34_생산계획_210405.csv', '6_P34_재고실적_210405.csv', '99_P34_판매계획_2021_01_02_03월_수정.csv', 'X_Product.csv', 'X_Sales.csv', 'Y_inven.csv']


In [4]:
#### 생산계획 ####
tmp_file = pd.read_csv('./data/'+f_list[4], thousands=',')
tmp_title = ['PLAN_START_YYYYMMDD','YYYYMMDD', 'QTY']
X_product_plan = tmp_file[tmp_title].copy() 
X_product_plan['PLAN_WW'] = pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d').dt.isocalendar().week
X_product_plan['PLAN_YEAR'] = pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d').dt.isocalendar().year
X_product_plan['TARGET_WW'] = pd.to_datetime(tmp_file['YYYYMMDD'], format='%Y%m%d').dt.isocalendar().week
X_product_plan['TARGET_YEAR'] = pd.to_datetime(tmp_file['YYYYMMDD'], format='%Y%m%d').dt.isocalendar().year

time_key = X_product_plan.drop_duplicates(['PLAN_YEAR', 'PLAN_WW','TARGET_YEAR','TARGET_WW'], keep='first')

cnt = 0
index_TY_ww = []
index_TY_year = []
for t in range(9, dt_info.shape[0]):
    QTY_sum = list()
    for j in range(t, t-8, -1):        
        PW = dt_info[j,1]; PY = dt_info[j,0]
        TW = dt_info[t,1]; TY = dt_info[t,0]                
        QTY_sum.append(X_product_plan[(X_product_plan['PLAN_YEAR']==PY) & (X_product_plan['PLAN_WW']==PW) & (X_product_plan['TARGET_YEAR']==TY) & (X_product_plan['TARGET_WW']==TW)]['QTY'].sum())
    if cnt== 0:
        tmp_X_product_plan = QTY_sum
    else:
        tmp_X_product_plan = np.vstack((tmp_X_product_plan, QTY_sum))
    cnt+=1
    if np.divmod(dt_info[t,1], 10)[0] == 0:
        index_TY_ww.append('W0'+str(dt_info[t,1]))
    else:
        index_TY_ww.append('W'+str(dt_info[t,1]))
    index_TY_year.append(TY)

X_product_plan = pd.DataFrame(tmp_X_product_plan).copy()
X_product_plan_title = list(map(lambda x : 'Product_Var'+str(x), list(range(1,9))))
X_product_plan.columns = X_product_plan_title
X_product_plan['YEAR'] = index_TY_year
X_product_plan['WW'] = index_TY_ww

#### 생산실적 ####
tmp_file = pd.read_csv('./data/'+f_list[3], encoding='euc-kr', thousands=',')
tmp_title = ['Prod Type','Material', 'Basic Date', 'Plan Date', 'Plan Qty', 'Result Qty']
tmp_p_real1 = tmp_file[tmp_title].copy()
tmp_p_real1['YEAR'] = pd.to_datetime(tmp_file['Plan Date']).dt.isocalendar().year
tmp_p_real1['WW'] = pd.to_datetime(tmp_file['Plan Date']).dt.isocalendar().week
X_product_actual = tmp_p_real1[(tmp_p_real1['Prod Type']=='MASS') & (tmp_p_real1['YEAR']==2020)& (tmp_p_real1['Material']=='APVCCCMA0-A2-A01')] 

cnt = 0
index_TY_ww = []
r_QTY_sum = list()
for t in range(9, dt_info.shape[0]):
    TW=dt_info[t,1]; TY=dt_info[t,0]    
    if any((X_product_actual['YEAR']==TY) & (X_product_actual['WW']==TW)):
        r_QTY_sum.append(X_product_actual[(X_product_actual['YEAR']==TY) & (X_product_actual['WW']==TW)]['Result Qty'].sum())
    else:
        r_QTY_sum.append(None)
X_product_plan['Product'] = r_QTY_sum
X_product_plan.to_csv('./data/X_Product.csv')

In [6]:
#### 재고실적 ####
tmp_file = pd.read_csv('./data/'+f_list[5], thousands=',')
tmp_title = ['DATE', 'MONTH', 'WW','DAY', 'IN_CELL', 'OUT_CELL', 'INVENTORY_CELL', 'IN_PACK_D', 'OUT_PACK_D', 'INVENTORY_PACK_D']                
tmp_Y_inven = tmp_file[tmp_title].copy()
tmp_Y_inven['YEAR'] = pd.to_datetime(tmp_file['DATE']).dt.year
Y_inven = tmp_Y_inven[['YEAR','WW', 'INVENTORY_PACK_D']].copy()
Y_set = Y_inven.groupby(['YEAR', 'WW']).mean()
Y_set.to_csv('./data/Y_inven.csv')

In [53]:
#### 판매계획/실적 Data Set ####
def column_merge(t_tmp, t_title):
    ww_unique = pd.Series.unique(t_title.iloc[0, 15:])
    t_tmp1 = t_tmp[t_title.iloc[0, :14]].copy()
    for j in range(0, len(ww_unique)):
        #t_tmp[ww_unique[j]] = t_tmp[ww_unique[j]].fillna(0)
        if len(t_tmp[ww_unique[j]].shape) > 1 :
            t_tmp1[ww_unique[j]] = t_tmp[ww_unique[j]].fillna(0).sum(axis=1)
        else:
            t_tmp1[ww_unique[j]] = t_tmp[ww_unique[j]].fillna(0)  
    return t_tmp1

tmp_title1 = pd.read_csv('./data/'+f_list[1], skiprows=0, nrows=1)#, header=None)
tmp1 = pd.read_csv('./data/'+f_list[1], skiprows=2, thousands=',')
tmp1.columns = tmp_title1.loc[0]
tmp_dta1 = column_merge(tmp1, tmp_title1)

tmp1_sub = pd.read_csv('./data/'+f_list[6], thousands=',')
#tmp_title2 = pd.read_csv('./data/'+f_list[2], skiprows=0, nrows=1)#, header=None)
#tmp2 = pd.read_csv('./data/'+f_list[2], skiprows=2, thousands=',')
#tmp2.columns = tmp_title2.loc[0]
#tmp_dta2 = column_merge(tmp2, tmp_title2)


In [54]:
cnt = 0 
index_ww =[]
index_year = []
tmp_dta_ww = pd.to_datetime(tmp_dta1['Version'].str.slice(start=5, stop=13), format='%Y%m%d').dt.isocalendar().week
for w in range(9, dt_info.shape[0]):
    if dt_info[w,0] == 2020:
        if np.divmod(dt_info[w,1],10)[0] == 0:
            tmp_index_ww = 'W0'+str(dt_info[w,1])
        else:
            tmp_index_ww = 'W'+str(dt_info[w,1])
        s_row = np.where(tmp_dta_ww == dt_info[w,1])[0][0]                
        tmp_x_var = tmp_dta1[tmp_index_ww].iloc[s_row:(s_row+8)]
        
        ww_gap = dt_info[w,1]-45
        if ww_gap <= 0:        
            tmp_post_ww = list(map(lambda x : 'W0'+str(x) if np.divmod(x, 10)[0] == 0 else 'W'+str(x), list(range(dt_info[w,1], dt_info[w,1]+8))))
            tmp_x_var_f = tmp_dta1[tmp_post_ww].iloc[s_row]
        else:
            tmp_post_ww = list(map(lambda x : 'W0'+str(x) if np.divmod(x, 10)[0] == 0 else 'W'+str(x), list(range(dt_info[w,1], dt_info[w,1]+8-ww_gap))))
            tmp_x_var_f = np.append(tmp_dta1[tmp_post_ww].iloc[s_row], np.repeat(0, ww_gap))
            
        if cnt == 0:
            tmp_x_sales_plan = tmp_x_var
            tmp_x_sales_plan_f = tmp_x_var_f
        else :
            tmp_x_sales_plan = np.vstack((tmp_x_sales_plan, tmp_x_var))
            tmp_x_sales_plan_f = np.vstack((tmp_x_sales_plan_f, tmp_x_var_f))
        cnt += 1
        index_ww.append(tmp_index_ww)
        index_year.append(dt_info[w,0])
#

In [55]:
cnt = 0 
index_ww =[]
index_year = []
tmp_dta_ww = pd.to_datetime(tmp_dta1['Version'].str.slice(start=5, stop=13), format='%Y%m%d').dt.isocalendar().week
for w in range(9, dt_info.shape[0]):
    if dt_info[w,0] == 2020:
        if np.divmod(dt_info[w,1],10)[0] == 0:
            tmp_index_ww = 'W0'+str(dt_info[w,1])
        else:
            tmp_index_ww = 'W'+str(dt_info[w,1])
        s_row = np.where(tmp_dta_ww == dt_info[w,1])[0][0]                
        tmp_x_var = tmp_dta1[tmp_index_ww].iloc[s_row:(s_row+8)]
        
        ww_gap = dt_info[w,1]-45
        if ww_gap <= 0:        
            tmp_post_ww = list(map(lambda x : 'W0'+str(x) if np.divmod(x, 10)[0] == 0 else 'W'+str(x), list(range(dt_info[w,1], dt_info[w,1]+8))))
            tmp_x_var_f = tmp_dta1[tmp_post_ww].iloc[s_row]
        else:
            tmp_post_ww = list(map(lambda x : 'W0'+str(x) if np.divmod(x, 10)[0] == 0 else 'W'+str(x), list(range(dt_info[w,1], dt_info[w,1]+8-ww_gap))))
            tmp_x_var_f = np.append(tmp_dta1[tmp_post_ww].iloc[s_row], np.repeat(0, ww_gap))
            
        if cnt == 0:
            tmp_x_sales_plan = tmp_x_var
            tmp_x_sales_plan_f = tmp_x_var_f
        else :
            tmp_x_sales_plan = np.vstack((tmp_x_sales_plan, tmp_x_var))
            tmp_x_sales_plan_f = np.vstack((tmp_x_sales_plan_f, tmp_x_var_f))
        cnt += 1
        index_ww.append(tmp_index_ww)
        index_year.append(dt_info[w,0])
        
X_sales_title = list(map(lambda x : 'Sales_Var_'+str(x), list(range(1,9))))
X_sales_title_f = list(map(lambda x : 'Sales_Var_Post_'+str(x), list(range(1,9))))

X_sales_plan = pd.DataFrame(np.concatenate((tmp_x_sales_plan, tmp_x_sales_plan_f),axis=1)) #pd.DataFrame(tmp_x_sales_plan)
X_sales_plan.columns = X_sales_title + X_sales_title_f
X_sales_plan['YEAR'] = index_year
X_sales_plan['WW'] = index_ww
X_sales_plan['Sales'] = tmp_dta1[index_ww].loc[0].values
X_sales_plan = X_sales_plan[X_sales_plan['WW'] !='W53'].copy()

for i in range(0, tmp1_sub.shape[0]):
    if np.divmod(tmp1_sub.iloc[i, 17], 10)[0] == 0:
        tmp1_sub.iloc[i,17] = 'W0'+str(tmp1_sub.iloc[i,17])
    else:
        tmp1_sub.iloc[i,17] = 'W'+str(tmp1_sub.iloc[i,17])

X_sales_plan = pd.concat([X_sales_plan, tmp1_sub])        
X_sales_plan.sort_values(['YEAR', 'WW'])
X_sales_plan.to_csv('./data/X_Sales.csv')

In [56]:
X_sales_plan

,Sales_Var_1,Sales_Var_2,Sales_Var_3,Sales_Var_4,Sales_Var_5,Sales_Var_6,Sales_Var_7,Sales_Var_8,Sales_Var_Post_1,Sales_Var_Post_2,Sales_Var_Post_3,Sales_Var_Post_4,Sales_Var_Post_5,Sales_Var_Post_6,Sales_Var_Post_7,Sales_Var_Post_8,YEAR,WW,Sales
0,504.0,504.0,504.0,504.0,504.0,504.0,588.0,588.0,504.0,588.0,84.0,420.0,672.0,588.0,672.0,1092.0,2020,W10,0.0
1,504.0,588.0,672.0,672.0,672.0,672.0,672.0,672.0,504.0,84.0,420.0,588.0,672.0,672.0,1092.0,756.0,2020,W11,0.0
2,84.0,84.0,84.0,672.0,672.0,672.0,1030.0,1030.0,84.0,420.0,672.0,588.0,672.0,924.0,756.0,934.0,2020,W12,0.0
3,0.0,420.0,420.0,420.0,1008.0,1008.0,1008.0,1260.0,0.0,0.0,0.0,336.0,588.0,1112.0,1112.0,1112.0,2020,W13,0.0
4,0.0,0.0,672.0,588.0,672.0,1092.0,1092.0,1092.0,0.0,0.0,0.0,596.0,672.0,1008.0,1260.0,1260.0,2020,W14,0.0
5,0.0,0.0,0.0,588.0,672.0,588.0,924.0,924.0,0.0,0.0,512.0,672.0,840.0,1176.0,756.0,1176.0,2020,W15,0.0
6,0.0,0.0,0.0,336.0,672.0,672.0,672.0,1124.0,0.0,92.0,588.0,840.0,1260.0,840.0,1176.0,1356.0,2020,W16,0.0
7,0.0,92.0,512.0,596.0,588.0,924.0,1092.0,1092.0,0.0,92.0,588.0,1176.0,840.0,1260.0,1020.0,1260.0,2020,W17,0.0
8,0.0,92.0,588.0,672.0,672.0,1112.0,756.0,756.0,0.0,420.0,840.0,924.0,1092.0,1104.0,1176.0,1188.0,2020,W18,-4.0
9,399.0,420.0,588.0,840.0,840.0,1008.0,1112.0,934.0,399.0,1008.0,672.0,1260.0,1008.0,1260.0,1260.0,1260.0,2020,W19,335.0
